In [2]:
!pip install ipycanvas ipywidgets


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from ipycanvas import Canvas, hold_canvas
from ipywidgets import VBox, HBox, Button, Output, Layout
import numpy as np
from IPython.display import display, clear_output

# --- 游戏参数 ---
BOARD_SIZE = 15
GRID_SIZE = 40
STONE_RADIUS = 16
BOARD_PIXEL = GRID_SIZE * (BOARD_SIZE - 1)

# --- 游戏状态 ---
board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=int)
current_player = 1
game_over = False

# --- 创建弹窗容器 ---
game_window = Output(layout=Layout(
    width='800px',
    height='700px',
    border='2px solid #ccc',
    padding='10px',
    background_color='#f9f9f9',
    margin='10px'
))

canvas = Canvas(width=BOARD_PIXEL+1, height=BOARD_PIXEL+1)
status_out = Output()

def draw_board():
    """绘制棋盘和棋子"""
    canvas.clear()
    # 棋盘底色
    canvas.fill_style = '#F0D9B5'
    canvas.fill_rect(0, 0, BOARD_PIXEL+1, BOARD_PIXEL+1)
    # 网格线
    canvas.stroke_style = '#333'
    canvas.line_width = 1
    for i in range(BOARD_SIZE):
        canvas.stroke_line(i*GRID_SIZE, 0, i*GRID_SIZE, BOARD_PIXEL)
        canvas.stroke_line(0, i*GRID_SIZE, BOARD_PIXEL, i*GRID_SIZE)
    # 星位（天元点）
    star_points = [3, 7, 11]
    for i in star_points:
        for j in star_points:
            canvas.fill_style = '#222'
            canvas.fill_circle(i*GRID_SIZE, j*GRID_SIZE, 4)
    # 绘制棋子
    for r in range(BOARD_SIZE):
        for c in range(BOARD_SIZE):
            if board[r, c] == 1:  # 黑子
                canvas.fill_style = 'black'
                canvas.fill_circle(c*GRID_SIZE, r*GRID_SIZE, STONE_RADIUS)
            elif board[r, c] == 2:  # 白子
                canvas.fill_style = 'white'
                canvas.stroke_style = 'black'
                canvas.line_width = 2
                canvas.fill_circle(c*GRID_SIZE, r*GRID_SIZE, STONE_RADIUS)
                canvas.stroke_circle(c*GRID_SIZE, r*GRID_SIZE, STONE_RADIUS)

def check_win(r, c, player):
    """检查是否获胜"""
    directions = [(1,0), (0,1), (1,1), (1,-1)]  # 横、竖、主对角线、副对角线
    for dr, dc in directions:
        count = 1
        # 向两个方向检查
        for d in [1, -1]:
            for i in range(1, 5):
                nr, nc = r + dr*i*d, c + dc*i*d
                if 0 <= nr < BOARD_SIZE and 0 <= nc < BOARD_SIZE and board[nr, nc] == player:
                    count += 1
                else:
                    break
        if count >= 5:
            return True
    return False

def handle_click(x, y):
    """处理鼠标点击"""
    global current_player, game_over
    if game_over:
        return
    # 计算棋盘坐标
    col = int(round(x / GRID_SIZE))
    row = int(round(y / GRID_SIZE))
    # 检查位置是否有效且为空
    if 0 <= row < BOARD_SIZE and 0 <= col < BOARD_SIZE and board[row, col] == 0:
        # 落子
        board[row, col] = current_player
        draw_board()
        # 检查是否获胜
        if check_win(row, col, current_player):
            with status_out:
                clear_output()
                print(f'🎉 恭喜！玩家 {"黑" if current_player==1 else "白"} 获胜！')
            game_over = True
            return
        # 切换玩家
        current_player = 3 - current_player
        with status_out:
            clear_output()
            print(f'当前轮到：{"黑" if current_player==1 else "白"}')

def on_canvas_click(x, y):
    """画布点击事件"""
    handle_click(x, y)

def reset_game(_=None):
    """重新开始游戏"""
    global board, current_player, game_over
    board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=int)
    current_player = 1
    game_over = False
    draw_board()
    with status_out:
        clear_output()
        print('当前轮到：黑')

def open_game_window(_=None):
    """打开游戏窗口"""
    with game_window:
        clear_output()
        # 创建控制按钮
        reset_btn = Button(description='🔄 重新开始', button_style='info', layout=Layout(width='120px'))
        close_btn = Button(description='❌ 关闭窗口', button_style='danger', layout=Layout(width='120px'))
        # 绑定按钮事件
        reset_btn.on_click(reset_game)
        close_btn.on_click(close_game_window)
        # 显示游戏界面
        display(VBox([
            canvas,
            HBox([reset_btn, close_btn], layout=Layout(justify_content='center')),
            status_out
        ]))
    # 绑定画布事件
    canvas.on_mouse_down(lambda x, y: on_canvas_click(x, y))
    # 初始化游戏
    draw_board()
    with status_out:
        print('当前轮到：黑')

def close_game_window(_=None):
    """关闭游戏窗口"""
    game_window.clear_output()

# --- 主控制面板 ---
open_btn = Button(
    description='🎮 打开五子棋游戏', 
    button_style='success', 
    layout=Layout(width='200px', height='50px', font_size='16px')
)
open_btn.on_click(open_game_window)

# --- 显示界面 ---
print('🎯 五子棋游戏控制面板')
print('=' * 30)
display(open_btn)
display(game_window)

🎯 五子棋游戏控制面板


Button(button_style='success', description='🎮 打开五子棋游戏', layout=Layout(height='50px', width='200px'), style=But…

Output(layout=Layout(border_bottom='2px solid #ccc', border_left='2px solid #ccc', border_right='2px solid #cc…

In [ ]:
# 实现了五子棋游戏的ipycanvas可视化界面，包括主控制面板、游戏窗口的打开与关闭、棋盘绘制、落子逻辑及相关按钮的交互功能。

import webbrowser
import tempfile
from pathlib import Path

# HTML 内容
html_code = """<!DOCTYPE html>
<html lang="zh">
<head>
  <meta charset="UTF-8">
  <title>五子棋</title>
  <style>
    body { text-align: center; font-family: sans-serif; }
    canvas { border: 1px solid #555; margin-top: 20px; }
  </style>
</head>
<body>
  <h2>🎮 五子棋 HTML Canvas 版</h2>
  <canvas id="board" width="600" height="600"></canvas><br><br>
  <button onclick="resetGame()">🔄 重新开始</button>
  <p id="status">当前轮到：黑棋</p>
<script>
const canvas = document.getElementById('board');
const ctx = canvas.getContext('2d');
const size = 15;
const gridSize = 40;
const radius = 16;
let board = Array.from({length: size}, () => Array(size).fill(0));
let currentPlayer = 1;
let gameOver = false;

function drawBoard() {
    ctx.fillStyle = "#F0D9B5";
    ctx.fillRect(0, 0, canvas.width, canvas.height);
    ctx.strokeStyle = "#333";
    for (let i = 0; i < size; i++) {
        ctx.beginPath();
        ctx.moveTo(gridSize * i, 0);
        ctx.lineTo(gridSize * i, canvas.height);
        ctx.stroke();
        ctx.beginPath();
        ctx.moveTo(0, gridSize * i);
        ctx.lineTo(canvas.width, gridSize * i);
        ctx.stroke();
    }
    const stars = [3, 7, 11];
    for (let i of stars) {
        for (let j of stars) {
            ctx.beginPath();
            ctx.arc(i * gridSize, j * gridSize, 4, 0, Math.PI * 2);
            ctx.fillStyle = "#222";
            ctx.fill();
        }
    }
    for (let r = 0; r < size; r++) {
        for (let c = 0; c < size; c++) {
            if (board[r][c] !== 0) {
                ctx.beginPath();
                ctx.arc(c * gridSize, r * gridSize, radius, 0, Math.PI * 2);
                ctx.fillStyle = board[r][c] === 1 ? "black" : "white";
                ctx.fill();
                ctx.strokeStyle = "black";
                ctx.stroke();
            }
        }
    }
}

function getMousePos(evt) {
    const rect = canvas.getBoundingClientRect();
    return {
        x: Math.round((evt.clientX - rect.left) / gridSize),
        y: Math.round((evt.clientY - rect.top) / gridSize)
    };
}

function checkWin(r, c, player) {
    const dirs = [[1,0], [0,1], [1,1], [1,-1]];
    for (let [dr, dc] of dirs) {
        let count = 1;
        for (let d of [-1, 1]) {
            for (let i = 1; i < 5; i++) {
                let nr = r + i * dr * d;
                let nc = c + i * dc * d;
                if (nr >= 0 && nr < size && nc >= 0 && nc < size && board[nr][nc] === player) {
                    count++;
                } else break;
            }
        }
        if (count >= 5) return true;
    }
    return false;
}

function handleClick(evt) {
    if (gameOver) return;
    const pos = getMousePos(evt);
    const r = pos.y;
    const c = pos.x;
    if (r < 0 || r >= size || c < 0 || c >= size || board[r][c] !== 0) return;
    board[r][c] = currentPlayer;
    drawBoard();
    if (checkWin(r, c, currentPlayer)) {
        document.getElementById("status").innerText = `🎉 玩家 ${currentPlayer === 1 ? '黑' : '白'} 获胜！`;
        gameOver = true;
    } else {
        currentPlayer = 3 - currentPlayer;
        document.getElementById("status").innerText = `当前轮到：${currentPlayer === 1 ? '黑棋' : '白棋'}`;
    }
}

function resetGame() {
    board = Array.from({length: size}, () => Array(size).fill(0));
    currentPlayer = 1;
    gameOver = false;
    drawBoard();
    document.getElementById("status").innerText = "当前轮到：黑棋";
}

canvas.addEventListener("click", handleClick);
drawBoard();
</script>
</body>
</html>
"""

# 写入临时 HTML 文件
html_path = Path(tempfile.gettempdir()) / "gomoku.html"
html_path.write_text(html_code, encoding='utf-8')

# 打开浏览器
webbrowser.open(f"file://{html_path}")


True